<a href="https://colab.research.google.com/github/CandidoJr209/Pytorch_examples/blob/main/Pytorch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MNIST digit classification with Convolutional Neural Network**


In this notebook we'll classify mnist digits using a CNN implemented with pytorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms


First step, build the model

In [2]:
class CNN(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.fc1 = nn.Linear(16*7*7,num_classes)

  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)

    return x

Now select the device to use, that will be a cuda device if you have one and a cpu otherwise

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Tune the hiperparameters

In [3]:
in_channels = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epoch = 1

Load the data

In [ ]:
train_dataset = datasets.MNIST(root='dataset/',train=True,transform=transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset = datasets.MNIST(root='dataset/',train=False,transform=transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

Now initialize the model with the parameters you chose

In [6]:
model = CNN().to(device)

Define the loss and optimizer functions

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Finally, train the model and display your results

In [7]:
for epoch in range(num_epoch):
  for batch_idx, (data, targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)


    scores = model(data)
    loss = loss_fn(scores,targets)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
    

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

  model.train()


check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Got 57833 / 60000 with accuracy 96.39
Got 9644 / 10000 with accuracy 96.44
